In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import os

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, MaxPooling2D, Activation
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/Research/Data"

Classifier  First_CNN_Training.ipynb  Test  Train  Untitled0.ipynb  Valid


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError("GPU device not found")
print("Found GPU at: {}".format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
img_height, img_width = 224, 224
batch_size = 5
classify_names = ['correct_masked', 'incorrect_masked', 'no_mask']

In [ ]:
image_path = '/content/drive/My Drive/Research/Data'
model_path = '/content/drive/My Drive/Ronald/models/'

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    image_path,
    labels='inferred',
    label_mode='int',
    # class_names=tf.train,
    validation_split=0.3,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 9431 files belonging to 4 classes.
Using 6602 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    image_path,
    labels='inferred',
    label_mode='int',
    # class_names=classify_names,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 9431 files belonging to 4 classes.
Using 1886 files for validation.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from keras.layers.serialization import activation
def get_model():
  resnet_model = Sequential()

  pretrained_model = tf.keras.applications.VGG16(include_top = False,
                                                    input_shape = (224, 224, 3),
                                                    pooling = 'avg',classes = 3,
                                                    weights='imagenet')
  for layer in pretrained_model.layers:
    layer.trainable=False

  resnet_model.add(pretrained_model)
  resnet_model.add(Flatten())
  resnet_model.add(Dense(512, activation = 'relu'))
  resnet_model.add(Dropout(0.3))
  resnet_model.add(Dense(3, activation = 'softmax'))
  resnet_model.compile(optimizer=Adam(learning_rate = 0.001),
                       loss='sparse_categorical_crossentropy',
                       metrics = ['accuracy'])

  resnet_model.summary()

  return resnet_model

"""
from tensorflow.keras.models import Sequential
def get_model():
  VGG_Model = Sequential()
  pretrained_model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3), pooling='avg', classes=3, weights='imagenet')
  for layer in pretrained_model.layers:
    layer.trainable = False
  VGG_Model.add(pretrained_model)
  VGG_Model.add(Flatten())
  VGG_Model.add(Dense(512, activation='relu'))
  VGG_Model.add(Dropout(0.3))
  VGG_Model.add(Dense(3, activation='softmax'))
  VGG_Model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
  VGG_Model.summary()
  return VGG_Model
"""

'\nfrom tensorflow.keras.models import Sequential\ndef get_model():\n  VGG_Model = Sequential()\n  pretrained_model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3), pooling=\'avg\', classes=3, weights=\'imagenet\')\n  for layer in pretrained_model.layers:\n    layer.trainable = False\n  VGG_Model.add(pretrained_model)\n  VGG_Model.add(Flatten())\n  VGG_Model.add(Dense(512, activation=\'relu\'))\n  VGG_Model.add(Dropout(0.3))\n  VGG_Model.add(Dense(3, activation=\'softmax\'))\n  VGG_Model.compile(optimizer="adam", loss=\'categorical_crossentropy\', metrics=[\'accuracy\'])\n  VGG_Model.summary()\n  return VGG_Model\n'

In [ ]:
from keras.api._v2.keras import callbacks

from IPython.core import history


with tf.device('/device:GPU:0'):
  resnet_model = get_model()

  history = resnet_model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=EarlyStopping(monitor='val_loss', patience=30))

from matplotlib import pyplot as plt
plt.plot(history.history['loss'], 'r', label = 'training loss')
plt.plot(history.history['val_loss'], label = 'validation loss')

plt.title('Model Loss')
plt.ylabel('Loss')

plt.xlabel('Epochs')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], 'r', label = "training accuracy")
plt.plot(history.history['val_accuracy'], label = "validation accuracy")

plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.show()

resnet_model.save(model_path+'ResNet_3_categories.h5')

58889256/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 14,978,883
Trainable params: 264,195
Non-trainable params: 14,714,688
_______________________________________

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss'], 'r', label = 'training loss')
plt.plot(history.history['val_loss'], label = 'validation loss')

plt.title('Model Loss')
plt.ylabel('Loss')

plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], 'r', label = "training accuracy")
plt.plot(history.history['val_accuracy'], label = "validation accuracy")

plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
resnet_model.save(model_path+'ResNet_3_categories.h5')

Load the Saved Trained Model and use it for  testing

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, MaxPooling2D, Activation
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.python.platform.test import test_src_dir_path
test_path = '/content/drive/My Drive/Research/Data/Test/'
test_model_path = '/content/drive/My Drive/Ronald/models/'


In [ ]:
from keras.models import load_model
from keras.layers import Dense, Flatten, Dropout, AveragePooling2D, Conv2D, Input
from keras.preprocessing import image

saved_model = load_model(test_model_path+'ResNet_3_categories.h5')

In [ ]:
import numpy as np
y_pred = [] #store predicted labels
y_true = [] #store true labels

for image_batch, label_batch in val_ds:
  y_true.append(label_batch)
  preds = saved_model.predict(image_batch)
  y_pred.append(np.argmax(preds, axis = -1))


In [ ]:
import tensorflow
true_categories = tensorflow.concat([item for item in y_true], axis = 0)
predicted_categories = tensorflow.concat([item for item in y_pred], axis = 0)

In [ ]:
from sklearn.metrics import confusion_matrix

cnf_matrix = confusion_matrix(true_categories, predicted_categories, normalize='pred')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=cnf_matrix,
                              display_labels=classify_names)

disp = disp.plot(cmap = 'Blues', xticks_rotation=45)

plt.show()
